In [12]:
############ ITALY - KML and KMZ FILES


import geopandas as gpd

gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Italy/Massarosa_2022/doc.kml', driver='KML')
print(gdf.crs) 
# Reproject to EPSG:32719 for UTM zone 19S)
gdf = gdf.to_crs(epsg=32719)
print(gdf.crs)

EPSG:4326
EPSG:32719


/home/polivera/miniconda3/envs/geo_data/lib/python3.10/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'doc.kml': 'Punti Bozzano' (default), 'Perimetro Incendio Bozzano'. Specify layer parameter to avoid this warning.
  result = read_func(


In [5]:
### CALCI
import geopandas as gpd
import pandas as pd
import re

gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Italy/Calci_2018/Evoluzione Perimetro Incendio Monte Serra 24 set 2018_00.kml', driver='KML')
gdf = gdf.to_crs(epsg=32719)

def extract_iso_timestamp(name):
    m = re.search(r'(\d{4})(\d{2})(\d{2})_(\d{2})(\d{2})', str(name))
    if m:
        return f"{m.group(1)}-{m.group(2)}-{m.group(3)} {m.group(4)}:{m.group(5)}"
    else:
        return None

# Create column "iso_timestamp" as text
gdf['iso_timestamp'] = gdf['Name'].apply(extract_iso_timestamp)


# Delete attributes
gdf = gdf.drop('description', axis=1)
gdf = gdf.drop('timestamp', axis=1)
gdf = gdf.drop('begin', axis=1)
gdf = gdf.drop('end', axis=1)
gdf = gdf.drop('drawOrder', axis=1)
gdf = gdf.drop('icon', axis=1)
gdf = gdf.drop('altitudeMode', axis=1)
gdf = gdf.drop('tessellate', axis=1)
gdf = gdf.drop('extrude', axis=1)
gdf = gdf.drop('visibility', axis=1)
gdf = gdf.drop('snippet', axis=1)
gdf = gdf.drop('Name', axis=1)


gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length 

# Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

gdf = gdf.sort_values('iso_timestamp')

gdf_2 = gdf.dissolve(by='iso_timestamp', as_index=False)

gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

gdf_2 = gdf_2[mand_atrib + extra_atrib]

# Save GeoJSON
gdf_2.to_file('CalciFire_test9.geojson', driver='GeoJSON')

In [33]:
### MASSAROSA - KMZ FILE
import geopandas as gpd
import zipfile
import os
import fiona

# Ruta al archivo KMZ
kmz_path = '/home/polivera/Documents/NERO - Database/Italy/Massarosa_2022/Evoluzione Perimetro Incendio Massarosa 2022.kmz'

# Carpeta temporal para extraer KML
temp_dir = '/home/polivera/Documents/NERO - Database/Italy/Massarosa_2022'


os.makedirs(temp_dir, exist_ok=True)

# Extraer todos los archivos del KMZ (que es un ZIP)
with zipfile.ZipFile(kmz_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Buscar archivo(s) KML extraído(s)
kml_files = [f for f in os.listdir(temp_dir) if f.lower().endswith('.kml')]

if not kml_files:
    raise FileNotFoundError("No se encontró archivo KML en el KMZ extraído.")

# Leer el primer archivo KML para trabajar
kml_path = os.path.join(temp_dir, kml_files[0])
gdf = gpd.read_file(kml_path, driver='KML')

layers = fiona.listlayers(kml_path)
for layer in layers:
    gdf = gpd.read_file(kml_path, driver='KML', layer=layer)
    print(f"Capa: {layer}")
    print(gdf.columns)

gdf.to_file('/home/polivera/Documents/NERO - Database/Italy/Massarosa_2022/mifle.shp')





Capa: Punti Bozzano
Index(['id', 'Name', 'description', 'timestamp', 'begin', 'end',
       'altitudeMode', 'tessellate', 'extrude', 'visibility', 'drawOrder',
       'icon', 'descriptio', 'giorno', 'ora', 'data', 'geometry'],
      dtype='object')
Capa: Perimetro Incendio Bozzano
Index(['id', 'Name', 'description', 'timestamp', 'begin', 'end',
       'altitudeMode', 'tessellate', 'extrude', 'visibility', 'drawOrder',
       'icon', 'snippet', 'geometry'],
      dtype='object')


/home/polivera/miniconda3/envs/geo_data/lib/python3.10/site-packages/pyogrio/geopandas.py:265: UserWarning: More than one layer found in 'doc.kml': 'Punti Bozzano' (default), 'Perimetro Incendio Bozzano'. Specify layer parameter to avoid this warning.
  result = read_func(
/tmp/ipykernel_413895/2224332031.py:36: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file('/home/polivera/Documents/NERO - Database/Italy/Massarosa_2022/mifle.shp')
/home/polivera/miniconda3/envs/geo_data/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'description' to 'descriptio'
  ogr_write(
/home/polivera/miniconda3/envs/geo_data/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field timestamp created as String field, though DateTime requested.
  ogr_write(
/home/polivera/miniconda3/envs/geo_data/lib/python3.10/site-packages/pyogrio/raw.py:723: RuntimeWarning: Field begin created as St

In [4]:
### MASSAROSA
import geopandas as gpd
import pandas as pd

# Rename attributes

gdf = gpd.read_file('/home/polivera/Documents/NERO - Database/Italy/Massarosa_2022/mifle.shp')
gdf = gdf.to_crs(epsg=32719)

gdf = gdf.rename(columns={
    'Name': 'iso_timestamp'
})

#Delete attributes
gdf = gdf.drop('descriptio', axis=1)
gdf = gdf.drop('timestamp', axis=1)
gdf = gdf.drop('begin', axis=1)
gdf = gdf.drop('end', axis=1)
gdf = gdf.drop('drawOrder', axis=1)
gdf = gdf.drop('icon', axis=1)
gdf = gdf.drop('altitudeMo', axis=1)
gdf = gdf.drop('tessellate', axis=1)
gdf = gdf.drop('extrude', axis=1)
gdf = gdf.drop('visibility', axis=1)
gdf = gdf.drop('snippet', axis=1)



# Add new attributes
gdf['area_ha'] = gdf.geometry.area / 10000  
gdf['perimeter_m'] = gdf.geometry.length

gdf = gdf.sort_values('iso_timestamp')

gdf_2 = gdf.dissolve(by='iso_timestamp', as_index=False)

gdf_2 = gdf_2.sort_values('iso_timestamp').reset_index(drop=True)
gdf_2['id'] = gdf_2.index + 1

mand_atrib= ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

extra_atrib = [col for col in gdf_2.columns if col not in mand_atrib]

gdf_2 = gdf_2[mand_atrib + extra_atrib]




# # Convierte la columna a datetime (si no lo está)
# gdf['iso_timestamp'] = pd.to_datetime(gdf['iso_timestamp'], errors='coerce')

# # Ordena de más antiguo a más reciente
# gdf = gdf.sort_values('iso_timestamp', ascending=True).reset_index(drop=True)
# gdf['id'] = gdf.index + 1

# columns_orden = ['id', 'iso_timestamp', 'area_ha', 'perimeter_m']

# Save GeoJSON
gdf_2.to_file('MassarosaFire_test13.geojson', driver='GeoJSON')
